# week 3 assignment 
This notebook will contain the third week assignemnt for the data sceince capstone course.

### First, I will install the required libraries for web scraping

In [83]:
!pip install bs4
!pip install requests 

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


#### I will import the libraries that I will use in web scraping. 

In [84]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import numpy as np

The url will be assinged to a variable 'url'

In [85]:
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=prev&oldid=926287641'

Data will then be pulled from the url and converted into text.

In [86]:
data = requests.get(url).text

A beautifulsoap object will be created and then I will retrieve all the tables using the find_all method.

In [87]:
soup = BeautifulSoup(data,"html5lib")
tables = soup.find_all('table')

The table of interest is extracted. 

In [88]:
pd.read_html(str(tables[0]), flavor='bs4')

[    Postcode           Borough          Neighbourhood
 0        M1A      Not assigned           Not assigned
 1        M2A      Not assigned           Not assigned
 2        M3A        North York              Parkwoods
 3        M4A        North York       Victoria Village
 4        M5A  Downtown Toronto           Harbourfront
 ..       ...               ...                    ...
 283      M8Z         Etobicoke              Mimico NW
 284      M8Z         Etobicoke     The Queensway West
 285      M8Z         Etobicoke  Royal York South West
 286      M8Z         Etobicoke         South of Bloor
 287      M9Z      Not assigned           Not assigned
 
 [288 rows x 3 columns]]

Then, I will convert the table from a list to a panda dataframe. 

In [89]:
table = pd.read_html(str(tables[0]), flavor='bs4')[0]
table

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West
286,M8Z,Etobicoke,South of Bloor


I will drop all the rows that have "not assigned" boroughs or neihbourhoods.

In [90]:
table = table[table["Borough"].str.contains("Not assigned")==False]
table = table[table["Neighbourhood"].str.contains("Not assigned")==False]
table = table.reset_index(drop = True)
table.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [91]:
table.shape

(210, 3)

#### The table of postal codes and neighbourhoods includes 210 rows and 3 columns.

Now, I will add the CSV file that contains the coordinates to a panda datafram.

In [92]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


I will rename the column to make it easier when combining the tables.

In [93]:
coordinates = coordinates.rename(columns = {'Postal Code': 'Postcode'})
coordinates.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


##### The two tables will be merged to create the desired dataframe for analysis.

In [94]:
ready_table = pd.merge(table, coordinates, how = "left", on = "Postcode")

In [95]:
ready_table.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


Analysis will be limited to the boroughs that conatain 'Toronto' in its name.

In [96]:
Toronto_data = ready_table[ready_table['Borough'].str.contains('Toronto', na = False)].reset_index(drop=True)
Toronto_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418


##### First, I will import the geolocation library to help me retrieve the latitude and longitude of Toronto city.

In [97]:
from geopy.geocoders import Nominatim

In [98]:
address = 'Toronto, TOR'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.7370584, -79.2442535.


##### Then, I will import the visualization libraries. 

In [99]:
import matplotlib.cm as cm
import matplotlib.colors as colors
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

# All requested packages already installed.



##### I will create a map of Toronto that have the neighbourhoods on it.

In [100]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Borough'], Toronto_data['Neighbourhood'] ):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Now, I will utilize the Foursquare API to explore the neighbourhoods and segment them. 

In [101]:
CLIENT_ID = 'B0TGUAOTILJSBOU0TQ0K0CFXXT0JXH3B1G4NDQR1P0Z4MPUC'
CLIENT_SECRET = 'VYUN4OU4CWSEFUGCR4XCFBQZ3GKVOTZY1UHCU4KT0OVJIE24'
VERSION = '20180605'
LIMIT = 100


##### First, I will define a function that get the nearby venues for each neighbourhood. 

In [102]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now, we get all the venues within neighbourhoods.

In [103]:
Toronto_venues = getNearbyVenues(names = Toronto_data['Neighbourhood'],
                                latitudes = Toronto_data['Latitude'],
                                longitudes = Toronto_data['Longitude']
                                 )

Harbourfront
Regent Park
Ryerson
Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
The Danforth West
Riverdale
Design Exchange
Toronto Dominion Centre
Brockton
Exhibition Place
Parkdale Village
The Beaches West
India Bazaar
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
Forest Hill West
High Park
The Junction South
North Toronto West
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
Harbord
University of Toronto
Runnymede
Swansea
Moore Park
Summerhill East
Chinatown
Grange Park
Kensington Market
Deer Park
Forest Hill SE
Rathnelly
South Hill
Summerhill West
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city


In [104]:
Toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,Harbourfront,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


Let's check how many venues are for each neoghbourhood.

In [105]:
Toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,15,15,15,15,15,15
Berczy Park,56,56,56,56,56,56
Brockton,24,24,24,24,24,24
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
...,...,...,...,...,...,...
Underground city,100,100,100,100,100,100
Union Station,100,100,100,100,100,100
University of Toronto,33,33,33,33,33,33


Let's check how many unique venues categories do we have.

In [106]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 236 uniques categories.


### Now, let's analyze each neighbourhood individually to see venues within and their categories. 

In [107]:
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

Toronto_onehot['Neighbourhood'] = Toronto_venues['Neighbourhood'] 


fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [108]:
Toronto_onehot.shape

(3093, 237)

Now, let's group the resulted dataframe by neighbourhood and take the mean and the frequency of each category. 

In [109]:
Toronto_grouped = Toronto_onehot.groupby('Neighbourhood').mean().reset_index()
Toronto_grouped

,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Adelaide,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.02,0.0,...,0.0,0.0,0.0,0.00,0.020000,0.000000,0.0,0.00,0.0,0.000000
1,Bathurst Quay,0.0,0.066667,0.066667,0.066667,0.133333,0.133333,0.133333,0.00,0.0,...,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.00,0.0,0.000000
2,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.0,0.0,0.00,0.017857,0.000000,0.0,0.00,0.0,0.000000
3,Brockton,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.00,0.0,0.041667
4,Business Reply Mail Processing Centre 969 Eastern,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.00,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,Underground city,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.03,0.0,...,0.0,0.0,0.0,0.01,0.010000,0.000000,0.0,0.01,0.0,0.000000
69,Union Station,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.0,0.0,0.01,0.010000,0.000000,0.0,0.01,0.0,0.000000
70,University of Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.0,0.0,0.00,0.000000,0.030303,0.0,0.00,0.0,0.030303
71,Victoria Hotel,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.03,0.0,...,0.0,0.0,0.0,0.00,0.020000,0.000000,0.0,0.01,0.0,0.000000


In [110]:
Toronto_grouped.shape

(73, 237)

##### Now, let's find the top five most common venues for each neighbourhood.

In [111]:
num_top_venues = 5

for hood in Toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
             venue  freq
0      Coffee Shop  0.09
1             Café  0.05
2   Clothing Store  0.05
3  Thai Restaurant  0.04
4       Restaurant  0.04


----Bathurst Quay----
              venue  freq
0    Airport Lounge  0.13
1   Airport Service  0.13
2  Airport Terminal  0.13
3   Harbor / Marina  0.07
4           Airport  0.07


----Berczy Park----
                venue  freq
0        Cocktail Bar  0.05
1              Bakery  0.05
2         Coffee Shop  0.05
3  Seafood Restaurant  0.04
4      Farmers Market  0.04


----Brockton----
            venue  freq
0            Café  0.12
1     Coffee Shop  0.08
2  Breakfast Spot  0.08
3          Bakery  0.08
4     Yoga Studio  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1          Comic Shop  0.06
2       Auto Workshop  0.06
3          Skate Park  0.06
4                 Spa  0.06


----CN Tower----
              venue  freq
0    Airport Lounge  0.13
1

#### Now, let's create a dataframe that have top 10 most common venues for each neighbourhood. 
First I will a create a function that sorts venues in descending order.

In [112]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [113]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = Toronto_grouped['Neighbourhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Clothing Store,Café,Hotel,Thai Restaurant,Restaurant,Gym,Pizza Place,Sushi Restaurant,Burrito Place
1,Bathurst Quay,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Plane,Airport,Airport Food Court,Airport Gate,Boutique,Harbor / Marina
2,Berczy Park,Bakery,Cocktail Bar,Coffee Shop,Seafood Restaurant,Restaurant,Beer Bar,Pharmacy,Cheese Shop,Farmers Market,Shopping Mall
3,Brockton,Café,Bakery,Breakfast Spot,Coffee Shop,Yoga Studio,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store
4,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Gym / Fitness Center,Garden,Brewery,Farmers Market,Spa,Fast Food Restaurant,Burrito Place,Restaurant,Auto Workshop


## Clustering the neighbourhoods 

First, I will import the kmeans module from the scikit learn library.

In [114]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [115]:
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighbourhood', 1)

Toronto_grouped_clustering = StandardScaler().fit_transform(Toronto_grouped_clustering)

kmeans = KMeans(init="k-means++", n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

kmeans.labels_[0:10] 

array([4, 0, 4, 2, 2, 0, 2, 4, 3, 2], dtype=int32)

Now, let's create a datafame that include the cluster and the top 10 venues for each neighbourhood. 

In [116]:
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_data

Toronto_merged = Toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

Toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,2,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Café,Theater,Yoga Studio,Farmers Market,Restaurant
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,2,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Café,Theater,Yoga Studio,Farmers Market,Restaurant
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,4,Coffee Shop,Clothing Store,Cosmetics Shop,Japanese Restaurant,Hotel,Bubble Tea Shop,Café,Ramen Restaurant,Bookstore,Pizza Place
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,4,Coffee Shop,Clothing Store,Cosmetics Shop,Japanese Restaurant,Hotel,Bubble Tea Shop,Café,Ramen Restaurant,Bookstore,Pizza Place
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,4,Coffee Shop,Café,Restaurant,Bakery,Italian Restaurant,Cosmetics Shop,Cocktail Bar,Clothing Store,Gastropub,Park


Now, let's visiualize the results. 

In [117]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Now, let'ts examine each cluster and determine what venues best describe this category. 

### Cluster 1

In [118]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,Downtown Toronto,0,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Plane,Airport,Airport Food Court,Airport Gate,Boutique,Harbor / Marina
60,Downtown Toronto,0,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Plane,Airport,Airport Food Court,Airport Gate,Boutique,Harbor / Marina
61,Downtown Toronto,0,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Plane,Airport,Airport Food Court,Airport Gate,Boutique,Harbor / Marina
62,Downtown Toronto,0,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Plane,Airport,Airport Food Court,Airport Gate,Boutique,Harbor / Marina
63,Downtown Toronto,0,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Plane,Airport,Airport Food Court,Airport Gate,Boutique,Harbor / Marina
64,Downtown Toronto,0,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Plane,Airport,Airport Food Court,Airport Gate,Boutique,Harbor / Marina
65,Downtown Toronto,0,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Plane,Airport,Airport Food Court,Airport Gate,Boutique,Harbor / Marina


#### We notice that the cluster 1 is characterized by airport and harbor venues.

### Cluster 2

In [119]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,West Toronto,1,Café,Mexican Restaurant,Thai Restaurant,Grocery Store,Furniture / Home Store,Bookstore,Fast Food Restaurant,Flea Market,Italian Restaurant,Speakeasy
37,West Toronto,1,Café,Mexican Restaurant,Thai Restaurant,Grocery Store,Furniture / Home Store,Bookstore,Fast Food Restaurant,Flea Market,Italian Restaurant,Speakeasy


#### Cluster 2 is characterized by having resturants and grocery stores.

### Cluster 3 

In [120]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,2,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Café,Theater,Yoga Studio,Farmers Market,Restaurant
1,Downtown Toronto,2,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Café,Theater,Yoga Studio,Farmers Market,Restaurant
5,East Toronto,2,Pub,Health Food Store,Neighborhood,Trail,Yoga Studio,Doner Restaurant,Discount Store,Distribution Center,Dog Run,Dumpling Restaurant
8,Downtown Toronto,2,Grocery Store,Café,Park,Italian Restaurant,Baby Store,Candy Store,Coffee Shop,Nightclub,Athletics & Sports,Restaurant
12,West Toronto,2,Bakery,Pharmacy,Supermarket,Bar,Middle Eastern Restaurant,Café,Music Venue,Bank,Gym / Fitness Center,Park
13,West Toronto,2,Bakery,Pharmacy,Supermarket,Bar,Middle Eastern Restaurant,Café,Music Venue,Bank,Gym / Fitness Center,Park
19,East Toronto,2,Greek Restaurant,Italian Restaurant,Coffee Shop,Café,Ice Cream Shop,Furniture / Home Store,Pub,Indian Restaurant,Caribbean Restaurant,Spa
20,East Toronto,2,Greek Restaurant,Italian Restaurant,Coffee Shop,Café,Ice Cream Shop,Furniture / Home Store,Pub,Indian Restaurant,Caribbean Restaurant,Spa
23,West Toronto,2,Café,Bakery,Breakfast Spot,Coffee Shop,Yoga Studio,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store
24,West Toronto,2,Café,Bakery,Breakfast Spot,Coffee Shop,Yoga Studio,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store


#### Cluster 3 is characterized by having entertainment facilities like pubs, cafes, and fitness centers. 

### Cluster 4

In [121]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,Downtown Toronto,3,Café,Bar,Coffee Shop,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Grocery Store,Park,Bakery,Caribbean Restaurant,Burger Joint
52,Downtown Toronto,3,Café,Bar,Coffee Shop,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Grocery Store,Park,Bakery,Caribbean Restaurant,Burger Joint
53,Downtown Toronto,3,Café,Bar,Coffee Shop,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Grocery Store,Park,Bakery,Caribbean Restaurant,Burger Joint


#### Cluster 4 is characterized by having bars, coffee shops,  and resturants. 

### Cluster 5 

In [122]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,4,Coffee Shop,Clothing Store,Cosmetics Shop,Japanese Restaurant,Hotel,Bubble Tea Shop,Café,Ramen Restaurant,Bookstore,Pizza Place
3,Downtown Toronto,4,Coffee Shop,Clothing Store,Cosmetics Shop,Japanese Restaurant,Hotel,Bubble Tea Shop,Café,Ramen Restaurant,Bookstore,Pizza Place
4,Downtown Toronto,4,Coffee Shop,Café,Restaurant,Bakery,Italian Restaurant,Cosmetics Shop,Cocktail Bar,Clothing Store,Gastropub,Park
6,Downtown Toronto,4,Bakery,Cocktail Bar,Coffee Shop,Seafood Restaurant,Restaurant,Beer Bar,Pharmacy,Cheese Shop,Farmers Market,Shopping Mall
7,Downtown Toronto,4,Coffee Shop,Café,Restaurant,Sandwich Place,Italian Restaurant,Salad Place,Japanese Restaurant,Bubble Tea Shop,Burger Joint,Miscellaneous Shop
9,Downtown Toronto,4,Coffee Shop,Clothing Store,Café,Hotel,Thai Restaurant,Restaurant,Gym,Pizza Place,Sushi Restaurant,Burrito Place
10,Downtown Toronto,4,Coffee Shop,Clothing Store,Café,Hotel,Thai Restaurant,Restaurant,Gym,Pizza Place,Sushi Restaurant,Burrito Place
11,Downtown Toronto,4,Coffee Shop,Clothing Store,Café,Hotel,Thai Restaurant,Restaurant,Gym,Pizza Place,Sushi Restaurant,Burrito Place
14,Downtown Toronto,4,Coffee Shop,Aquarium,Hotel,Café,Fried Chicken Joint,Scenic Lookout,Brewery,Restaurant,Italian Restaurant,Sports Bar
15,Downtown Toronto,4,Coffee Shop,Aquarium,Hotel,Café,Fried Chicken Joint,Scenic Lookout,Brewery,Restaurant,Italian Restaurant,Sports Bar


#### Cluster 4 is characterized by having cafes and restaurants. 